In [1]:
import os, sys
from python_tools import *

python tools loaded.


In [14]:
#take in hadron list, and calculate basic properties
#return theta_xz, theta_yz, theta_z, ke
def calc_hadrons(pdg=None,px=None,py=None,pz=None,energy=None,s=None):
    
    if s is not None:
        pdg = s[0]
        px = s[1]
        py = s[2]
        pz = s[3]
        energy = s[4]

    thetaxz = np.arctan2(px,pz)
    thetayz = np.arctan2(py,pz)
    thetaz = np.arctan2(np.sqrt(np.square(px)+np.square(py)),pz)
    ke = np.array( [ (energy[i] - particle.Particle.from_pdgid(pdg[i]).mass*1e-3) for i in range(len(pdg)) ],
                   dtype=np.dtype(np.float) )
    
    return thetaxz,thetayz,thetaz,ke

In [13]:
#utility function to get the max element of a sub-array
def get_max_val_by_mask(val_arr,mask):
    sub_idx = np.argmax(val_arr[mask])
    max_idx = np.arange(val_arr.shape[0])[mask][sub_idx]
    return max_idx

In [4]:
#apply an acceptance based on particle type, angle, and KE thresholds
#return acceptance for hadron list, number of passing hadrons, and properties of leading (by KE) hadron
def acceptance(pdg=None,px=None,py=None,pz=None,energy=None,thetaz=None,ke=None,
               pdgcode=2212,thetaz_thresh_deg=40,ke_thresh_mev=100.,
               s=None):
    
    if s is not None:
        pdg = s[0]
        px = s[1]
        py = s[2]
        pz = s[3]
        energy = s[4]
        thetaz = s[5]
        ke = s[6]
    
    #put in rad and GeV
    THETAZ_THRESH_RAD=np.radians(thetaz_thresh_deg)
    KE_THRESH_GEV=ke_thresh_mev*1e-3

    mask = (pdg==pdgcode)
    accept = np.logical_and(mask,np.abs(thetaz)<THETAZ_THRESH_RAD)
    
    px_1=np.nan
    py_1=np.nan
    pz_1=np.nan
    E_1=np.nan
    ke_1=np.nan
    thetaz_1=np.nan
    if np.any(accept):
        max_idx = get_max_val_by_mask(ke,accept)
        px_1=px[max_idx]
        py_1=py[max_idx]
        pz_1=pz[max_idx]
        E_1=energy[max_idx]
        ke_1=ke[max_idx]
        thetaz_1=thetaz[max_idx]
            
    return accept,np.sum(np.logical_and(accept,ke>KE_THRESH_GEV)),px_1,py_1,pz_1,E_1,ke_1,thetaz_1

In [28]:
#function for mergning in event weights
def merge_weights(file,tree_names,var_suffix,df):
    for i in range(len(tree_names)):
        w_df = uproot.open(file)[tree_names[i]].pandas.df()
        wght_name="wght_"+var_suffix[i]
        w_df = w_df.rename(columns={"wght": wght_name})
        df = df.merge(w_df[["iev",wght_name]],how="left",on=["iev"])
    return df

In [22]:
#function for combining all the event weights from the root files I've made...
def merge_weights_simple(filepath,knob_name,df):
    filename="%s/weights_%s.root"%(filepath,knob_name)
    tree_names = [ "%s_twk%d"%(knob_name,i) for i in [0,1,3,4] ]
    var_suffixes = [ "%s_n2"%knob_name,"%s_n1"%knob_name,"%s_p1"%knob_name,"%s_p2"%knob_name ]
    return merge_weights(filename,tree_names,var_suffixes,df)

In [7]:
#functions for some lepton quantities
def add_lepton_calcs(df):
    df['ptl'] = np.sqrt(np.square(df['pxl'])+np.square(df['pyl']))
    df['thetal'] = np.arctan2(df['ptl'],df['pzl'])

In [11]:
#functions for performing hadron calcs and acceptances
def add_hadron_calcs(df):
    df['thetaxzf'], df['thetayzf'], df['thetazf'], df['kef'] \
        = zip(*df[["pdgf","pxf","pyf","pzf","Ef"]].apply((lambda x: calc_hadrons(s=x)),axis=1))
    
def add_neutron_acceptance(df):
    df['accept_n'], df['n_n'], df['px_n1'], df['py_n1'], df['pz_n1'], df['E_n1'], df['ke_n1'], df['thetaz_n1'] \
        = zip(*df[["pdgf","pxf","pyf","pzf","Ef","thetazf","kef"]].apply((lambda x: acceptance(s=x,
                                                                                               pdgcode=2112,
                                                                                               thetaz_thresh_deg=40.,
                                                                                               ke_thresh_mev=500.)),
                                                                         axis=1))

def add_proton_acceptance(df):
    df['accept_p'], df['n_p'], df['px_p1'], df['py_p1'], df['pz_p1'], df['E_p1'], df['ke_p1'], df['thetaz_p1'] \
        = zip(*df[["pdgf","pxf","pyf","pzf","Ef","thetazf","kef"]].apply((lambda x: acceptance(s=x,
                                                                                               pdgcode=2212,
                                                                                               thetaz_thresh_deg=40.,
                                                                                               ke_thresh_mev=100.)),
                                                                         axis=1))


In [17]:
def create_base_df(root_filename,out_name=None,max_entries=-1,verbose=True):
    
    print("Reading %d entries from root file %s"%(max_entries,root_filename))
    gst_df = uproot.open(root_filename)['gst'].pandas.df(flatten=False)
    
    if(max_entries>0):
        gst_df = gst_df.loc[:max_entries]
    
    print("Adding lepton quantities.")
    #lepton quantities
    add_lepton_calcs(gst_df)
    
    print("Adding hadron quantities.")
    #hadron quantities
    add_hadron_calcs(gst_df)
    
    print("Adding hadron acceptances.")
    #acceptances
    add_neutron_acceptance(gst_df)
    add_proton_acceptance(gst_df)
    
    if(out_name):
        print("Writing output file %s"%out_name)
        gst_df.to_hdf(out_name,key="gst_df")
    
    return gst_df

In [35]:
gst_df = create_base_df("/Users/wketchum/Data/LDMX/gntp.0.gst.root",
                        max_entries=1e4)

Reading 10000 entries from root file /Users/wketchum/Data/LDMX/gntp.0.gst.root
Adding lepton quantities.
Adding hadron quantities.


/Library/Python/3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Adding hadron acceptances.


In [36]:
knob_names = ["FrCEx_N","FrAbs_N","FrInel_N","FrPiProd_N","MFP_N",
              "FrCEx_pi","FrAbs_pi","FrInel_pi","FrPiProd_pi","MFP_pi",
              "FormZone"]

In [37]:
for kn in knob_names:
    gst_df = merge_weights_simple(filepath="/Users/wketchum/Data/LDMX",knob_name=kn,df=gst_df)

In [38]:
gst_df

,iev,neu,fspl,tgt,Z,A,hitnuc,hitqrk,resid,sea,...,wght_FrPiProd_pi_p1,wght_FrPiProd_pi_p2,wght_MFP_pi_n2,wght_MFP_pi_n1,wght_MFP_pi_p1,wght_MFP_pi_p2,wght_FormZone_n2,wght_FormZone_n1,wght_FormZone_p1,wght_FormZone_p2
0,0,11,11,1000220480,22,48,2112,0,0,False,...,1.0,1.0,1.079574,1.044989,0.950952,0.901745,1.000000,1.000000,1.000000,1.000000
1,1,11,11,1000220480,22,48,2212,0,-99,False,...,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,2,11,11,1000220480,22,48,2112,0,0,False,...,1.0,1.0,1.516311,1.205978,0.853719,0.744598,1.000000,1.000000,1.000000,1.000000
3,3,11,11,1000220480,22,48,2212,0,6,False,...,1.0,1.0,1.000002,1.000002,0.999985,0.999913,1.000000,1.000000,1.000000,1.000000
4,4,11,11,1000220480,22,48,2212,0,-99,False,...,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,9996,11,11,1000220480,22,48,2212,2,-99,True,...,1.0,1.0,1.918825,1.366735,0.748789,0.571723,0.133834,0.462020,1.013492,0.493620
9997,9997,11,11,1000220480,22,48,2212,0,-99,False,...,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
9998,9998,11,11,1000220480,22,48,2212,0,-99,False,...,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
9999,9999,11,11,1000220480,22,48,2212,0,-99,False,...,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [39]:
gst_df.to_hdf("/Users/wketchum/Data/LDMX/base_generation_test.hdf","gst_df")

/Library/Python/3.8/site-packages/pandas/core/generic.py:2434: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block4_values] [items->Index(['pdgi', 'resc', 'Ei', 'pxi', 'pyi', 'pzi', 'pdgf', 'Ef', 'pxf', 'pyf',
       'pzf', 'pf', 'cthf', 'thetaxzf', 'thetayzf', 'thetazf', 'kef',
       'accept_n', 'accept_p'],
      dtype='object')]

  pytables.to_hdf(
